<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction
Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [1]:
!pip install sqlalchemy==1.3.9


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 78.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... one
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp312-cp312-linux_x86_64.whl size=1160111 sha256=e12dd507ad4860eb21daaa9a73859442a302631e0b437a750f1c4a4683302e9c
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/b3/1c/42/0e26b8d512adc6bce10ff71a05229366b4ccec641cd3b42111
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.37
    Uninstalling SQLAlchemy-2.0.37:
      Successfully uninstalled SQLAlchemy-2.0.37
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterhub 5.2.1 requires SQLAlchemy>=1.4.1, but you have sqlalchemy 1.3.9 which is incompatible.


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [2]:
!pip install ipython-sql
!pip install ipython-sql prettytable

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 84.9 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9


In [3]:
%load_ext sql

In [4]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [5]:
!pip install -q pandas

In [6]:
%sql sqlite:///my_data1.db

In [7]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

**Note:This below code is added to remove blank rows from table**


In [8]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [9]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

## Tasks

Now write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"**

### Task 1




##### Display the names of the unique launch sites  in the space mission


In [12]:
%%sql
SELECT DISTINCT "Launch_Site" 
FROM space_mission 
WHERE "Launch_Site" IS NOT NULL 
ORDER BY "Launch_Site";

 * sqlite:///my_data1.db
(sqlite3.OperationalError) no such table: space_mission
[SQL: SELECT DISTINCT "Launch_Site" 
FROM space_mission 
WHERE "Launch_Site" IS NOT NULL 
ORDER BY "Launch_Site";]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [13]:
conn = sqlite3.connect('my_data1.db')

# Get list of all tables
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, conn)
print("Tables in the database:")
print(tables)

conn.close()

Tables in the database:
          name
0    SPACEXTBL
1  SPACEXTABLE


In [14]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('my_data1.db')

# Option 1: Try SPACEXTBL first
query = """
SELECT DISTINCT "Launch_Site" 
FROM SPACEXTBL 
WHERE "Launch_Site" IS NOT NULL 
ORDER BY "Launch_Site";
"""

try:
    df = pd.read_sql_query(query, conn)
    print(f"Found {len(df)} unique launch sites in SPACEXTBL:")
    print(df.head())
    print("\nFull list:")
    print(df)
except Exception as e:
    print(f"Error with SPACEXTBL: {e}")
    
    # Option 2: Try SPACEXTABLE
    query = """
    SELECT DISTINCT "Launch_Site" 
    FROM SPACEXTABLE 
    WHERE "Launch_Site" IS NOT NULL 
    ORDER BY "Launch_Site";
    """
    
    try:
        df = pd.read_sql_query(query, conn)
        print(f"\nFound {len(df)} unique launch sites in SPACEXTABLE:")
        print(df.head())
        print("\nFull list:")
        print(df)
    except Exception as e2:
        print(f"Error with SPACEXTABLE: {e2}")
        
        # Let's find the correct column name
        print("\n=== Let's find the right column ===")
        for table in ['SPACEXTBL', 'SPACEXTABLE']:
            print(f"\nColumns in {table}:")
            columns_df = pd.read_sql_query(f"PRAGMA table_info({table});", conn)
            print(columns_df[['name', 'type']])

conn.close()

Found 4 unique launch sites in SPACEXTBL:
    Launch_Site
0   CCAFS LC-40
1  CCAFS SLC-40
2    KSC LC-39A
3   VAFB SLC-4E

Full list:
    Launch_Site
0   CCAFS LC-40
1  CCAFS SLC-40
2    KSC LC-39A
3   VAFB SLC-4E



### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [16]:

conn = sqlite3.connect('my_data1.db')

# Task 2: Display 5 records where launch sites begin with 'CCA'
query = """
SELECT * 
FROM SPACEXTBL 
WHERE "Launch_Site" LIKE 'CCA%' 
LIMIT 5;
"""

df = pd.read_sql_query(query, conn)
print(f"Found {len(df)} records where Launch_Site begins with 'CCA':")
print(df)

# Show all columns clearly
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
print("\nDetailed view:")
print(df)

conn.close()

Found 5 records where Launch_Site begins with 'CCA':
         Date Time (UTC) Booster_Version  Launch_Site  \
0  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  PAYLOAD_MASS__KG_  \
0               Dragon Spacecraft Qualification Unit                  0   
1  Dragon demo flight C1, two CubeSats, barrel of...                  0   
2                              Dragon demo flight C2                525   
3                                       SpaceX CRS-1                500   
4                                       SpaceX CRS-2                677   

       Orbit         Customer Mission_Outcome      Landing_Outcome  
0        LEO           SpaceX         Success  Failure (parachute)  
1  LEO (ISS)  N

### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [17]:
conn = sqlite3.connect('my_data1.db')

# First, let's check what columns are available in the table
print("=== Checking table structure ===")
columns_query = "PRAGMA table_info(SPACEXTBL);"
columns_df = pd.read_sql_query(columns_query, conn)
print(columns_df[['name', 'type']].to_string(index=False))

# Based on the column names, let's look for relevant columns
print("\n=== Checking for NASA/CRS related data ===")
print("Unique values in possible mission/customer columns:")

# Check for columns that might contain NASA/CRS information
possible_cols = ['Customer', 'Mission', 'Orbit', 'Launch_Site']
for col in possible_cols:
    if col in columns_df['name'].values:
        query = f'SELECT DISTINCT "{col}" FROM SPACEXTBL WHERE "{col}" IS NOT NULL LIMIT 10;'
        try:
            sample = pd.read_sql_query(query, conn)
            print(f"\nColumn: {col}")
            print(sample.head(10).to_string(index=False))
        except:
            pass

conn.close()

=== Checking table structure ===
             name    type
             Date    TEXT
       Time (UTC)    TEXT
  Booster_Version    TEXT
      Launch_Site    TEXT
          Payload    TEXT
PAYLOAD_MASS__KG_ INTEGER
            Orbit    TEXT
         Customer    TEXT
  Mission_Outcome    TEXT
  Landing_Outcome    TEXT

=== Checking for NASA/CRS related data ===
Unique values in possible mission/customer columns:

Column: Customer
                Customer
                  SpaceX
         NASA (COTS) NRO
             NASA (COTS)
              NASA (CRS)
                     MDA
                     SES
                 Thaicom
                 Orbcomm
                 AsiaSat
U.S. Air Force NASA NOAA

Column: Orbit
      Orbit
        LEO
  LEO (ISS)
  Polar LEO
        GTO
        HEO
        SSO
        MEO
Sub-orbital

Column: Launch_Site
 Launch_Site
 CCAFS LC-40
 VAFB SLC-4E
  KSC LC-39A
CCAFS SLC-40


### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [18]:
conn = sqlite3.connect('my_data1.db')

# Compare all booster versions
query = """
SELECT 
    "Booster_Version",
    AVG("Payload_Mass") as "Average_Payload_Mass",
    COUNT(*) as "Mission_Count"
FROM SPACEXTBL 
WHERE "Booster_Version" IS NOT NULL
GROUP BY "Booster_Version"
ORDER BY "Average_Payload_Mass" DESC;
"""

df = pd.read_sql_query(query, conn)
print("Average payload mass by booster version:")
print(df.to_string(index=False))

conn.close()

Average payload mass by booster version:
Booster_Version  Average_Payload_Mass  Mission_Count
  F9 v1.1 B1018                   0.0              1
  F9 v1.1 B1017                   0.0              1
  F9 v1.1 B1016                   0.0              1
  F9 v1.1 B1015                   0.0              1
  F9 v1.1 B1014                   0.0              1
  F9 v1.1 B1013                   0.0              1
  F9 v1.1 B1012                   0.0              1
  F9 v1.1 B1011                   0.0              1
  F9 v1.1 B1010                   0.0              1
 F9 v1.1  B1003                   0.0              1
        F9 v1.1                   0.0              5
 F9 v1.0  B0007                   0.0              1
 F9 v1.0  B0006                   0.0              1
 F9 v1.0  B0005                   0.0              1
 F9 v1.0  B0004                   0.0              1
 F9 v1.0  B0003                   0.0              1
  F9 FT B1038.1                   0.0              1
    F

### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_ 


In [19]:
conn = sqlite3.connect('my_data1.db')

# Task: Find the date of first successful ground pad landing
# We need to look for successful ground pad outcomes and find the earliest date

query = """
SELECT 
    MIN("Date") as "First_Successful_Ground_Pad_Landing_Date",
    "Landing_Outcome",
    "Booster_Version",
    "Launch_Site"
FROM SPACEXTBL 
WHERE ("Landing_Outcome" LIKE '%Success%' AND 
       ("Landing_Outcome" LIKE '%ground%' OR "Landing_Outcome" LIKE '%pad%'))
   OR "Landing_Outcome" = 'Success (ground pad)'
   OR "Landing_Outcome" = 'Success (ground pad).'
GROUP BY "Landing_Outcome"
ORDER BY "Date" ASC
LIMIT 1;
"""

try:
    df = pd.read_sql_query(query, conn)
    print("First successful ground pad landing:")
    print(df.to_string(index=False))
    
    if not df.empty:
        print(f"\nThe first successful ground pad landing was on: {df['First_Successful_Ground_Pad_Landing_Date'].iloc[0]}")
    else:
        print("No successful ground pad landings found with those criteria")
        
except Exception as e:
    print(f"Error: {e}")

conn.close()

First successful ground pad landing:
First_Successful_Ground_Pad_Landing_Date      Landing_Outcome Booster_Version Launch_Site
                              2015-12-22 Success (ground pad)     F9 FT B1019 CCAFS LC-40

The first successful ground pad landing was on: 2015-12-22


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [23]:
conn = sqlite3.connect('my_data1.db')

# Task 6: List boosters with success in drone ship AND payload mass > 4000 AND < 6000
query = """
SELECT DISTINCT 
    "Booster_Version",
    "Payload_Mass",
    "Landing_Outcome",
    "Date",
    "Launch_Site"
FROM SPACEXTBL 
WHERE "Landing_Outcome" LIKE '%Success%drone%'
   OR "Landing_Outcome" LIKE '%drone%Success%'
   OR "Landing_Outcome" LIKE '%drone ship%'
   OR "Landing_Outcome" = 'Success (drone ship)'
   OR "Landing_Outcome" = 'Success (drone ship).'
   AND "Payload_Mass" > 4000 
   AND "Payload_Mass" < 6000
ORDER BY "Booster_Version", "Date";
"""

print("Boosters with successful drone ship landings and payload mass 4000-6000:")
try:
    df = pd.read_sql_query(query, conn)
    if not df.empty:
        print(df.to_string(index=False))
        print(f"\nTotal unique boosters found: {df['Booster_Version'].nunique()}")
    else:
        print("No records found with exact criteria")
        
except Exception as e:
    print(f"Error: {e}")

conn.close()

Boosters with successful drone ship landings and payload mass 4000-6000:
Booster_Version "Payload_Mass"        Landing_Outcome       Date  Launch_Site
  F9 B4 B1041.1   Payload_Mass   Success (drone ship) 2017-10-09  VAFB SLC-4E
  F9 B4 B1042.1   Payload_Mass   Success (drone ship) 2017-10-30   KSC LC-39A
  F9 B4 B1045.1   Payload_Mass   Success (drone ship) 2018-04-18 CCAFS SLC-40
 F9 B5  B1046.1   Payload_Mass   Success (drone ship) 2018-05-11   KSC LC-39A
 F9 FT  B1021.2   Payload_Mass   Success (drone ship) 2017-03-30   KSC LC-39A
 F9 FT  B1029.2   Payload_Mass   Success (drone ship) 2017-06-23   KSC LC-39A
 F9 FT  B1031.2   Payload_Mass   Success (drone ship) 2017-10-11   KSC LC-39A
    F9 FT B1020   Payload_Mass   Failure (drone ship) 2016-03-04  CCAFS LC-40
  F9 FT B1021.1   Payload_Mass   Success (drone ship) 2016-04-08  CCAFS LC-40
    F9 FT B1022   Payload_Mass   Success (drone ship) 2016-05-06  CCAFS LC-40
  F9 FT B1023.1   Payload_Mass   Success (drone ship) 2016-05-27  CCA

### Task 7




##### List the total number of successful and failure mission outcomes


In [24]:
conn = sqlite3.connect('my_data1.db')

# Direct answer for Task 7
query = """
SELECT 
    'Success' as Mission_Outcome,
    COUNT(*) as Count
FROM SPACEXTBL 
WHERE UPPER("Mission_Outcome") LIKE '%SUCCESS%'

UNION ALL

SELECT 
    'Failure' as Mission_Outcome,
    COUNT(*) as Count
FROM SPACEXTBL 
WHERE UPPER("Mission_Outcome") LIKE '%FAIL%';
"""

df = pd.read_sql_query(query, conn)
print("Task 7: Total number of successful and failure mission outcomes")
print("=" * 60)
print(df.to_string(index=False))

# Calculate percentages
total_missions_query = "SELECT COUNT(*) as total FROM SPACEXTBL WHERE \"Mission_Outcome\" IS NOT NULL;"
total_df = pd.read_sql_query(total_missions_query, conn)
total = total_df['total'].iloc[0]

if total > 0:
    success_count = df[df['Mission_Outcome'] == 'Success']['Count'].iloc[0]
    failure_count = df[df['Mission_Outcome'] == 'Failure']['Count'].iloc[0]
    
    print(f"\nAdditional Statistics:")
    print(f"Total missions with outcomes: {total}")
    print(f"Success rate: {success_count/total*100:.1f}%")
    print(f"Failure rate: {failure_count/total*100:.1f}%")
    
    # Check for other outcomes
    other_count = total - success_count - failure_count
    if other_count > 0:
        print(f"Other outcomes: {other_count} ({other_count/total*100:.1f}%)")

conn.close()

Task 7: Total number of successful and failure mission outcomes
Mission_Outcome  Count
        Success    100
        Failure      1

Additional Statistics:
Total missions with outcomes: 101
Success rate: 99.0%
Failure rate: 1.0%


### Task 8



##### List all the booster_versions that have carried the maximum payload mass, using a subquery with a suitable aggregate function.


In [26]:
conn = sqlite3.connect('my_data1.db')

print("=== Different approaches to solve Task 8 ===")

print("\n1. Using subquery in WHERE clause:")
query1 = """
SELECT DISTINCT "Booster_Version"
FROM SPACEXTBL 
WHERE "Payload_Mass" = (SELECT MAX("Payload_Mass") FROM SPACEXTBL);
"""
df1 = pd.read_sql_query(query1, conn)
print(df1.to_string(index=False))

conn.close()

=== Different approaches to solve Task 8 ===

1. Using subquery in WHERE clause:
Booster_Version
 F9 v1.0  B0003
 F9 v1.0  B0004
 F9 v1.0  B0005
 F9 v1.0  B0006
 F9 v1.0  B0007
 F9 v1.1  B1003
        F9 v1.1
  F9 v1.1 B1011
  F9 v1.1 B1010
  F9 v1.1 B1012
  F9 v1.1 B1013
  F9 v1.1 B1014
  F9 v1.1 B1015
  F9 v1.1 B1016
  F9 v1.1 B1018
    F9 FT B1019
  F9 v1.1 B1017
    F9 FT B1020
  F9 FT B1021.1
    F9 FT B1022
  F9 FT B1023.1
    F9 FT B1024
  F9 FT B1025.1
    F9 FT B1026
  F9 FT B1029.1
  F9 FT B1031.1
    F9 FT B1030
 F9 FT  B1021.2
  F9 FT B1032.1
    F9 FT B1034
  F9 FT B1035.1
 F9 FT  B1029.2
  F9 FT B1036.1
    F9 FT B1037
  F9 B4 B1039.1
  F9 FT B1038.1
  F9 B4 B1040.1
  F9 B4 B1041.1
 F9 FT  B1031.2
  F9 B4 B1042.1
 F9 FT  B1035.2
 F9 FT  B1036.2
  F9 B4 B1043.1
 F9 FT  B1032.2
 F9 FT  B1038.2
    F9 B4 B1044
 F9 B4  B1041.2
 F9 B4  B1039.2
  F9 B4 B1045.1
 F9 B5  B1046.1
 F9 B4  B1043.2
 F9 B4  B1040.2
  F9 B4 B1045.2
   F9 B5B1047.1
   F9 B5B1048.1
  F9 B5 B1046.2
   F9 B

### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**


In [27]:
conn = sqlite3.connect('my_data1.db')

# Clean version for Task 9
query = """
SELECT 
    CASE substr("Date", 6, 2)
        WHEN '01' THEN 'January'
        WHEN '02' THEN 'February'
        WHEN '03' THEN 'March'
        WHEN '04' THEN 'April'
        WHEN '05' THEN 'May'
        WHEN '06' THEN 'June'
        WHEN '07' THEN 'July'
        WHEN '08' THEN 'August'
        WHEN '09' THEN 'September'
        WHEN '10' THEN 'October'
        WHEN '11' THEN 'November'
        WHEN '12' THEN 'December'
    END as "Month",
    "Landing_Outcome",
    "Booster_Version",
    "Launch_Site"
FROM SPACEXTBL 
WHERE substr("Date", 1, 4) = '2015'
  AND "Landing_Outcome" LIKE '%drone%'
  AND ("Landing_Outcome" LIKE '%Fail%' OR "Landing_Outcome" LIKE '%fail%')
ORDER BY substr("Date", 6, 2);
"""

print("Task 9: Month names, failure landing outcomes in drone ship, booster versions, launch_site for 2015")
print("=" * 100)

df = pd.read_sql_query(query, conn)

if not df.empty:
    print(df.to_string(index=False))
    
    # Show a count by month
    print(f"\n=== Count of drone ship failure landings by month in 2015 ===")
    month_counts = df['Month'].value_counts().sort_index(key=lambda x: pd.to_datetime(x, format='%B').dt.month)
    for month, count in month_counts.items():
        print(f"{month}: {count} failure(s)")
else:
    print("No records found matching the criteria.")
    
    # Try a broader search
    print("\nTrying broader search for drone ship outcomes in 2015...")
    broader_query = """
    SELECT 
        CASE substr("Date", 6, 2)
            WHEN '01' THEN 'January'
            WHEN '02' THEN 'February'
            WHEN '03' THEN 'March'
            WHEN '04' THEN 'April'
            WHEN '05' THEN 'May'
            WHEN '06' THEN 'June'
            WHEN '07' THEN 'July'
            WHEN '08' THEN 'August'
            WHEN '09' THEN 'September'
            WHEN '10' THEN 'October'
            WHEN '11' THEN 'November'
            WHEN '12' THEN 'December'
        END as "Month",
        "Landing_Outcome",
        "Booster_Version",
        "Launch_Site"
    FROM SPACEXTBL 
    WHERE substr("Date", 1, 4) = '2015'
      AND ("Landing_Outcome" LIKE '%drone%' OR "Landing_Outcome" LIKE '%DS%' OR "Landing_Outcome" LIKE '%ship%')
    ORDER BY substr("Date", 6, 2), "Landing_Outcome";
    """
    
    broader_df = pd.read_sql_query(broader_query, conn)
    if not broader_df.empty:
        print(f"\nFound {len(broader_df)} drone ship outcomes in 2015 (all types):")
        print(broader_df.to_string(index=False))

conn.close()

Task 9: Month names, failure landing outcomes in drone ship, booster versions, launch_site for 2015
  Month      Landing_Outcome Booster_Version Launch_Site
January Failure (drone ship)   F9 v1.1 B1012 CCAFS LC-40
  April Failure (drone ship)   F9 v1.1 B1015 CCAFS LC-40

=== Count of drone ship failure landings by month in 2015 ===


AttributeError: 'DatetimeIndex' object has no attribute 'dt'

### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [28]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('my_data1.db')

# Alternative without window functions
query = """
SELECT 
    "Landing_Outcome",
    COUNT(*) as Outcome_Count
FROM SPACEXTBL 
WHERE "Landing_Outcome" IS NOT NULL
  AND "Date" >= '2010-06-04' 
  AND "Date" <= '2017-03-20'
GROUP BY "Landing_Outcome"
ORDER BY Outcome_Count DESC, "Landing_Outcome";
"""

print("Task 10: Count of landing outcomes between 2010-06-04 and 2017-03-20")
print("=" * 100)

df = pd.read_sql_query(query, conn)

if not df.empty:
    # Add rank manually in Python
    df['Rank'] = range(1, len(df) + 1)
    
    # Reorder columns
    df = df[['Rank', 'Landing_Outcome', 'Outcome_Count']]
    
    print(df.to_string(index=False))
    
    # Show summary
    print(f"\n=== Summary ===")
    print(f"Total different landing outcomes: {len(df)}")
    print(f"Total landings in period: {df['Outcome_Count'].sum()}")
    
    # Show examples of specific outcomes mentioned in task
    print(f"\n=== Specific Outcome Examples ===")
    specific_outcomes = ['Failure (drone ship)', 'Success (ground pad)', 'Success (drone ship)', 'Failure (ground pad)']
    for outcome in specific_outcomes:
        matches = df[df['Landing_Outcome'].str.contains(outcome.replace('(', '\(').replace(')', '\)'), regex=False)]
        if not matches.empty:
            for _, row in matches.iterrows():
                print(f"{row['Landing_Outcome']}: {row['Outcome_Count']} occurrences (Rank {row['Rank']})")
        else:
            # Try partial match
            partial_matches = df[df['Landing_Outcome'].str.contains(outcome.split(' ')[0], case=False, na=False)]
            if not partial_matches.empty:
                for _, row in partial_matches.iterrows():
                    print(f"{row['Landing_Outcome']}: {row['Outcome_Count']} occurrences (Rank {row['Rank']})")
    
else:
    print("No landing outcomes found in the specified date range.")

conn.close()

Task 10: Count of landing outcomes between 2010-06-04 and 2017-03-20
 Rank        Landing_Outcome  Outcome_Count
    1             No attempt             10
    2   Failure (drone ship)              5
    3   Success (drone ship)              5
    4     Controlled (ocean)              3
    5   Success (ground pad)              3
    6    Failure (parachute)              2
    7   Uncontrolled (ocean)              2
    8 Precluded (drone ship)              1

=== Summary ===
Total different landing outcomes: 8
Total landings in period: 31

=== Specific Outcome Examples ===
Failure (drone ship): 5 occurrences (Rank 2)
Failure (parachute): 2 occurrences (Rank 6)
Success (drone ship): 5 occurrences (Rank 3)
Success (ground pad): 3 occurrences (Rank 5)
Success (drone ship): 5 occurrences (Rank 3)
Success (ground pad): 3 occurrences (Rank 5)
Failure (drone ship): 5 occurrences (Rank 2)
Failure (parachute): 2 occurrences (Rank 6)


### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>




## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


<!--
## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2024-07-10 | 1.1 |Anita Verma | Changed Version|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |
-->


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
